In [168]:
# these libraries for visualization
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go


In [169]:
# we work on only train beacues its already huge
train= pd.read_csv('/kaggle/input/digit-recognizer/train.csv')

In [170]:
train

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Its an image classification data so no need to do EDA and  scaling because its all are [0,1] columns,only we have to reduce the dimensionality through PCA

In [171]:
# lets see one row how our image is looking
train.sample()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
20228,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [172]:
# lets take this row
px.imshow(train.iloc[20417,1:].values.reshape(28,28))

In [173]:
# so its seems like to be the same number

In [174]:
# as it is classification problem lets do train-test-split
from sklearn.model_selection import train_test_split

In [175]:
x = train.iloc[:,1:]
y = train['label']

In [176]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.2,random_state=42)

In [177]:
# trying a model without doing pca than we will compare the accuracy after pca 
from sklearn.neighbors import KNeighborsClassifier

In [178]:
# lets try with three neighbors and check the accuracy
KNN =KNeighborsClassifier()

In [179]:
# shapes of our data
print('xtrain',xtrain.shape)
print('ytest',xtest.shape)
print('ytrain',ytrain.shape)
print('ytest',ytest.shape)

xtrain (33600, 784)
ytest (8400, 784)
ytrain (33600,)
ytest (8400,)


In [180]:
model =KNN.fit(xtrain,ytrain)

In [181]:
%%time 
pred = model.predict(xtest)

CPU times: user 33.4 s, sys: 2.98 s, total: 36.4 s
Wall time: 13.8 s


In [182]:
# lets check metrics that is working well or not
from sklearn.metrics import accuracy_score,classification_report


In [183]:
accuracy_score(ytest,pred)

0.9648809523809524

In [184]:
print(classification_report(ytest,pred))
# is we can see its performing well 

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       816
           1       0.94      1.00      0.97       909
           2       0.99      0.94      0.96       846
           3       0.96      0.96      0.96       937
           4       0.98      0.97      0.98       839
           5       0.96      0.95      0.96       702
           6       0.97      0.99      0.98       785
           7       0.96      0.96      0.96       893
           8       0.99      0.93      0.96       835
           9       0.94      0.95      0.95       838

    accuracy                           0.96      8400
   macro avg       0.97      0.96      0.96      8400
weighted avg       0.97      0.96      0.96      8400



In [185]:
from sklearn.model_selection import KFold,cross_val_score
kfold = KFold(n_splits=5,shuffle=True)
cross_val_score(KNN,xtrain,ytrain,cv = kfold,scoring = 'accuracy')

array([0.9639881 , 0.96235119, 0.9578869 , 0.96622024, 0.9671131 ])

In [186]:
cross_val_score(KNN,xtest,ytest,cv = kfold,scoring = 'accuracy')

array([0.93988095, 0.94166667, 0.93511905, 0.94404762, 0.93630952])

**we are geeting a perfect accuracy without doing pca that means our model isworking well**


- lets reduce dimensionlity and than check inertaction between our labels

In [187]:
from sklearn.preprocessing import StandardScaler
sc= StandardScaler()

In [188]:
xtrain = sc.fit_transform(xtrain)
xtest = sc.fit_transform(xtest)

In [189]:
from sklearn.decomposition import PCA

In [190]:
pca= PCA(n_components=2)

In [191]:
xtrain_2d= pd.DataFrame(pca.fit_transform(xtrain),columns =['PCA1','PCA2'])
xtest_2d= pd.DataFrame(pca.fit_transform(xtest),columns =['PCA1','PCA2'])

In [192]:
kmeans = KNeighborsClassifier()

In [193]:
p = kmeans.fit(xtrain,ytrain).predict(xtest)

In [194]:
accuracy_score(ytest,p)

0.9407142857142857

In [195]:
# as we can see our accuracy has reduced after applying PCA without PCA our our knn model 
# is working well just to visualize we have taken out  dimension
ytrain = ytrain.astype('str')

In [199]:
pt.scatter(x= xtrain_2d['PCA1'],y =xtrain_2d['PCA2'],color = ytrain)

### In 3 Dimensional


In [200]:
pca= PCA(n_components=3)

In [201]:
xtrain_3d= pd.DataFrame(pca.fit_transform(xtrain),columns =['PCA1','PCA2','PCA3'])
xtest_3d= pd.DataFrame(pca.fit_transform(xtest),columns =['PCA1','PCA2','PCA3'])

In [203]:
pt.scatter_3d(x= xtrain_3d['PCA1'],y =xtrain_3d['PCA2'],z = xtrain_3d['PCA3'],color = ytrain)

In [ ]:
pd.read_csv('/kaggle/input/digit-recognizer/sample_submission.csv')